In [1]:
%matplotlib widget
import caesar
import numpy as np
import matplotlib.pyplot as plt
import scipy 

/Users/ondrea/anaconda3/lib/python3.8/_collections_abc.py:666: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  self[key]


 In this notebook I have several methods for cleaning and cutting galaxy data before determining the cause of quenching.

In [2]:
obj = caesar.load('/Users/ondrea/Downloads/caesar_296.hdf5') # 296=redshift0

yt : [INFO     ] 2021-08-31 13:59:31,429 Opening /Users/ondrea/Downloads/caesar_296.hdf5
yt : [INFO     ] 2021-08-31 13:59:31,657 Found 54135 halos
yt : [INFO     ] 2021-08-31 13:59:31,701 Found 6480 galaxies


In [3]:
sfr = np.array([gal.sfr for gal in obj.galaxies])   # I want to cut data by SFR and mass
stellar_masses = np.array([gal.masses['stellar'] for gal in obj.galaxies])

In [4]:
stellar_masses[stellar_masses == 0] = 1e-99  #for log
sfr[sfr == 0] = 1e-99

In [5]:
stellar_masses

array([1.05960459e+12, 8.26104807e+11, 7.95258585e+11, ...,
       2.64359616e+08, 2.61941984e+08, 2.61884624e+08])

In [6]:
#Make a col of specific star formation rate (SFR/gal stellar mass)*(1e9 Gyr/yr)
ssfr = (sfr/stellar_masses)*1e9 

Since the science question is "what is the cause of quenching in high mass galaxies?" I need to remove low mass ones. I'll start with log(M/Msun) = 9.5

In [7]:
high_mass_idx = np.where(np.log10(stellar_masses) >= 9.5)

In [8]:
stellar_masses = stellar_masses[high_mass_idx]
ssfr = ssfr[high_mass_idx]

In [9]:
from sklearn.mixture import GaussianMixture

In [10]:
X = np.zeros([len(ssfr), 2])
for i in range(len(ssfr)):
    X[i,0] = ssfr[i]
    X[i,1] = stellar_masses[i]

In [11]:
X

array([[9.43748267e-103, 1.05960459e+012],
       [1.21050016e-102, 8.26104807e+011],
       [1.25745263e-102, 7.95258585e+011],
       ...,
       [5.95354143e-002, 3.17521024e+009],
       [3.05556799e-001, 3.17212723e+009],
       [5.23080001e-001, 3.17007898e+009]])

In [23]:
gmm = GaussianMixture(n_components=2, covariance_type='diag',random_state=1).fit(X)

In [24]:
ifig=2;plt.close(ifig);plt.figure(ifig)
#plt.scatter(gal_data.stellar_masses[labels == 1], np.log10(gal_data.ssfr[labels == 1]), s=4, alpha=0.4, c='b');
#plt.scatter(gal_data.stellar_masses[labels == 0], np.log10(gal_data.ssfr[labels == 0]), s=4, alpha=0.4, c='k');
plt.scatter(np.log10(X[:, 1]), np.log10(X[:, 0]), c=labels, s=4, cmap='viridis');
plt.xlabel('log stellar mass')
plt.ylabel('log ssfr')
plt.xlim(9.5,12);
plt.ylim(-4,1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
gmm?